# Slice Gui

In [ ]:
%pylab ipympl

In [ ]:
import os
import sys
import inspect
import logging
import ipywidgets as widgets
from ppmpy import ppm
print(inspect.getfile(ppm))

# turn off matplotlib messages
logging.getLogger("matplotlib").setLevel(logging.CRITICAL)

The next cell will scan the ASDR data and find runs that have the required Moms and Rprof data needed for the Moms GUI. This takes a moment. If you know what you want you can just hardwire the directory variable `rprof_dir` and `moms_dir` and skip this step. Else select a run in the drop-down menu and don't forget to press the _Select Runs_ button! The scanner may find runs it thinks it can render because data files are present. The scanner does not check for physical data integrity and if the correct values are 

In [ ]:
base_dir  = '/data/ASDR/PPMstar/'
root_dirs = os.listdir(base_dir)
options =[]

for sub_dir in root_dirs:
    if (os.path.isdir(base_dir + sub_dir)):
        data_dirs = os.listdir(base_dir + sub_dir)
        for folder in data_dirs:
            if (os.path.isdir(base_dir + sub_dir + '/' + folder)):
                full_dir = base_dir + sub_dir + '/' + folder
                if (os.path.exists(full_dir + '/prfs') and os.path.exists(full_dir + '/myavsbq')):
                    if ((os.listdir(full_dir + '/prfs') != []) and (os.listdir(full_dir + '/myavsbq') != [])):
                        options.append(full_dir)

select_dropdown = widgets.Dropdown(options=options, description="Path Select", layout=widgets.Layout(width='450px'))
select_button = widgets.Button(description="Select Runs", disabled=False)
selected_label = widgets.Label(value="")
select_group = widgets.VBox([select_dropdown, select_button, selected_label])

def select(b):
    global rprof_dir
    global moms_dir
    rprof_dir = select_dropdown.value + '/prfs/'
    moms_dir = select_dropdown.value + '/myavsbq/'
    selected_label.value = select_dropdown.value
select_button.on_click(select)

display(select_group)

In [ ]:
# A tested case: N15 is a RAWD simulation on a 768^3 grid (Stephens et al. 2020)
rprof_dir = "/data/ASDR/PPMstar/LowZRAWD/N15-LowZRAWD-768-10x-burn-moms/prfs/"
moms_dir  = "/data/ASDR/PPMstar/LowZRAWD/N15-LowZRAWD-768-10x-burn-moms/myavsbq/"

## Slice Gui: How to use guide
It is advisable to start with looking at the radial profiles using the `RProf_gui.ipynb` notebook to get an overview of the run.

1) Select the path to the data you wish to examine in dropdown above\
2) Click the `Select Runs` button above\
3) Run the block below to initialize the `Slice Gui` for those runs of data\

There are 5 sets of controls below in the `Graph Settings` tab, they are the following:
- Dump, selects the dump number of the data set
- Slice, selects the location of the slice on the selected Slice plane. For 192 cubed data for example, the middle of the star will be at Slice 91
- Quantity, there are 10 quantities examinable in this GUI. To see which quanties these are see line 11769 of ppm.py
- Slice plane, axis in which to take slices upon
- Log Value Range/Value Range. Log Value Range applies a limit to the maximum and minimum values logged within the selected data symetrically about the zero and then offsets by log10(min). Value Range just limits the range of unlogged data to show, this will automatically go to min/max of the selected data when switched to from log or when the quantity is switched.
- the uses as default `interpolation='kaiser'` which smoothes the pixelation a bit. You may not want that and use `interpolation=None`. This is an argument to [Matplotlib `imshow`](https://matplotlib.org/3.3.2/api/_as_gen/matplotlib.pyplot.imshow.html).

In [ ]:
#get rprof and moms data
rprof_data = ppm.RprofSet(rprof_dir)
moms_data = ppm.MomsDataSet(moms_dir, 100, rprofset=rprof_data)

moms_data.slice_gui()